In [1]:
import pandas as pd
import os
import numpy as np
from string_search import *

data_dir = r'C:\Users\ozano\Desktop\senet'
data_path = os.path.join(data_dir, 'results_me.csv')

In [2]:
df = pd.read_csv(data_path, sep = ';')
cols_to_use = ['AD', 'ADRES']

df = df[cols_to_use]
df.shape

(1000, 2)

In [3]:
"""df = pd.concat([df for _ in range(2300)], axis = 0)
df.shape"""

'df = pd.concat([df for _ in range(2300)], axis = 0)\ndf.shape'

## Preprocess

In [4]:
for col in df.columns:
    df[col] = df[col].apply(lambda x: preprocess(x))

"""df.loc[df.index == 0, 'AD'] = 'zeynel'
df.loc[df.index == 2, 'AD'] = 'zeynel abakayli'
df.loc[df.index == 3, 'AD'] = 'zeynel bakaysdsfdlik ad'"""
df.head(10)

,AD,ADRES
0,mehmet kandemir,palazoglu sok n2 sisli
1,mehmet halil elceoglu,demirtas mah deveoglu sok tikde is mrk no 33...
2,meryem yildiz,yesilkent mah 1807 sk no 233 esenyurt
3,metsan sosyal hizmetler limited sirketi,orhantepe mah tokdemir sokak no 101 kartal
4,mehmet keser,baris mah akdeniz cadde beyaz center avm 126...
5,med egitimdersane yayincilik basimm pazarlama ...,hayat sokak no 30 sariyer
6,mehmet ali gedik,baskopru mah d100 karayolu cad no 208 serdivan
7,mehmet nurullah altas,bagcilar mah alinak sitesi otopark karsisi n...
8,mese kimya danismanlik makina san ve tic ltd sti,sepetlipinar mah demokratlar cd no 16 basiskele
9,mesut tufekcioglu,yildiztepe mah 829 sokak no 1924 altindag


## Get N-Grams

In [5]:
df_ngram = pd.DataFrame()
for col in df.columns:
    df_ngram[col] = df[col].apply(lambda x: get_n_grams(x))

In [6]:
df_ngram.head()

,AD,ADRES
0,"[meh, ehm, hme, met, ka, kan, and, nde, dem, ...","[pal, ala, laz, azo, zog, ogl, glu, so, sok, ..."
1,"[meh, ehm, hme, met, ha, hal, ali, lil, el, ...","[dem, emi, mir, irt, rta, tas, ma, mah, de, ..."
2,"[mer, ery, rye, yem, yi, yil, ild, ldi, diz]","[yes, esi, sil, ilk, lke, ken, ent, ma, mah, ..."
3,"[met, ets, tsa, san, so, sos, osy, sya, yal, ...","[orh, rha, han, ant, nte, tep, epe, ma, mah, ..."
4,"[meh, ehm, hme, met, ke, kes, ese, ser]","[bar, ari, ris, ma, mah, ak, akd, kde, den, ..."


## Create Index

In [7]:
#%%timeit 85.8 ms

import math

def get_n_gram_length(x):
    return max(1, len(x))

n_index_tokens = np.array([4, 7, 10, 15, 20, 30, 50, 70])
labels = ['AD', 'ADRES']

index_data = {label: {'index':[], 'vocabulary':[], 'lengths': []} for label in labels}

for n_index_token in n_index_tokens:
    for label in labels:
        index, vocab, lengths = create_ngram_index_sparse(df_ngram[label].values, first_n_tokens = n_index_token)
        index_data[label]['index'].append(index)
        index_data[label]['vocabulary'].append(vocab)
        index_data[label]['lengths'].append(lengths)

## Search

In [8]:
s = 'mehmet caliskan ahmet doger'
input_n_gram_set = reduce_n_grams(input_n_grams)

len([t for t in input_n_gram_set if t in reduce_n_grams(get_n_grams(s))]) / len(input_n_gram_set)

NameError: name 'input_n_grams' is not defined

In [10]:
input_string = 'mehmet calik'
search_label = 'AD'
search_person = True

# mehmet kocamanoglu erkan calik
# mehmet caliskan ahmet doger
# reneva otomotiv insaat gida turizm tasimacilik sanayi ve tic
# metin aydinhusamettin aydin
#search_person = False # Person: True, Company: False, Address: False

input_n_grams = get_n_grams(input_string)
n_ngrams = len(input_n_grams)
#index_size = np.argmin(np.abs(n_index_tokens - n_ngrams))
index_size = len(n_index_tokens) - 1

if not search_person:
    for i, n_index_token in enumerate(n_index_tokens[:-1]):
        if n_ngrams <= n_index_token:
            index_size = i
            break

print(f'Input n_gram count: {n_ngrams}')
print(f'Searching with index size: {n_index_tokens[index_size]}')

search_data = (index_data[search_label]['index'][index_size], index_data[search_label]['vocabulary'][index_size], index_data[search_label]['lengths'][-1])
match_groups = search_groups_vectorized(input_n_grams, search_data[0], search_data[1], search_data[2],
    get_sorted = True, search_person = search_person)

search_count = 0
while len(match_groups[1.0]) == 0 and len(match_groups[0.8]) == 0 and index_size < len(n_index_tokens) - 1 and not search_person:

    index_size = index_size + 1
    print(f'Searching with index size: {n_index_tokens[index_size]}')

    search_data = (index_data[search_label]['index'][index_size], index_data[search_label]['vocabulary'][index_size], index_data[search_label]['lengths'][-1])
    match_groups = search_groups_vectorized(input_n_grams, search_data[0], search_data[1], search_data[2],
        get_sorted = True, search_person = search_person)

Input n_gram count: 10
Searching with index size: 70


In [11]:
match_values = get_match_values(match_groups, df[search_label].values)

## Get values

In [9]:
match_values[1.0]

NameError: name 'match_values' is not defined

In [12]:
match_values[0.8]

[(107, 'mehmet caliskan', 0.9),
 (209, 'mehmet caliskan', 0.9),
 (987, 'mehmet kocamanoglu erkan calik', 0.9),
 (988, 'mehmet kocamanoglu erkan calik', 0.9),
 (989, 'mehmet kocamanoglu erkan calik', 0.9)]

In [13]:
match_values[0.6]

[(63, 'mehmet cuneyt emlik', 0.7),
 (82, 'mehmet celik', 0.7),
 (128, 'mehmet salih edebali', 0.7),
 (140, 'mehmet cakmak', 0.7),
 (150, 'mehmet kascali', 0.7),
 (162, 'mehmet cakir', 0.7),
 (172, 'mehmet can', 0.7),
 (190, 'mehmet cat', 0.7),
 (328, 'mehmet cakar', 0.7),
 (359, 'mehmet cakir', 0.7),
 (436, 'mehmet cuneyt emlik', 0.7),
 (560, 'mehmet ali tahtali', 0.7),
 (612, 'mehmet can akinci', 0.7),
 (667, 'mehmet celik', 0.7),
 (737, 'mehmet celik', 0.7),
 (758, 'mehmet sehmuz can', 0.7),
 (797, 'mehmet cakmak', 0.7),
 (809, 'mehmet can degerci', 0.7),
 (828, 'mehmet hikmet barutcu', 0.7),
 (902, 'mehmet alikursat ozcan', 0.7),
 (1, 'mehmet halil elceoglu', 0.6),
 (6, 'mehmet ali gedik', 0.6),
 (11, 'mehmet nuri alim', 0.6),
 (18, 'mehmet ali yuksel', 0.6),
 (34, 'mehmet ali emirbayer', 0.6),
 (60, 'mehmet necdet narin', 0.6),
 (74, 'mehmet coskunca', 0.6),
 (83, 'mehmet culcu', 0.6),
 (101, 'mehmet ali sag', 0.6),
 (102, 'mehmet ceylan', 0.6),
 (176, 'mehmet cevizci', 0.6),
 (183

In [23]:
match_values[0.4]

[(0, 'mehmet kandemir', 0.5),
 (4, 'mehmet keser', 0.5),
 (7, 'mehmet nurullah altas', 0.5),
 (10, 'mehmet zeki dogan', 0.5),
 (12, 'mehmet duran', 0.5),
 (13, 'mehmet serkan konur', 0.5),
 (17, 'mehmet yaylaci', 0.5),
 (20, 'mehmet emin bayar', 0.5),
 (21, 'mehmet temucin yuksel', 0.5),
 (22, 'mehmet kizginel', 0.5),
 (26, 'mehmet duman', 0.5),
 (27, 'mehmet karatas', 0.5),
 (28, 'mehmet dogru', 0.5),
 (29, 'mehmet adir', 0.5),
 (30, 'mehmet okur', 0.5),
 (36, 'mehmet muhittin turgut', 0.5),
 (37, 'mehmet emin yildirim', 0.5),
 (38, 'mehmet muhittin turgut', 0.5),
 (39, 'mehmet muhittin turgut', 0.5),
 (40, 'mehmet muhittin turgut', 0.5),
 (41, 'mehmet muhittin turgut', 0.5),
 (42, 'mehmet karagoz', 0.5),
 (43, 'mehmet akturk', 0.5),
 (45, 'mehmet bayer', 0.5),
 (46, 'mehmet akay', 0.5),
 (47, 'mehmet yildiz', 0.5),
 (49, 'mehmet sekkeli', 0.5),
 (50, 'mehmet zahir gonultas', 0.5),
 (53, 'mehmet aydin', 0.5),
 (54, 'mehmet sekkeli', 0.5),
 (55, 'mehmet yilmaz', 0.5),
 (56, 'mehmet yap

In [31]:
match_values[0.0][:25]

[(0, 'mehmet kandemir', 0.38461538461538464),
 (22, 'mehmet kizginel', 0.38461538461538464),
 (72, 'mehmet kizginel', 0.38461538461538464),
 (155, 'mehmet tekinkus', 0.38461538461538464),
 (179, 'mehmet demirhan', 0.38461538461538464),
 (180, 'mehmet kayaturk', 0.38461538461538464),
 (193, 'mehmet agcaabat', 0.38461538461538464),
 (207, 'mehmet karakaya', 0.38461538461538464),
 (214, 'mehmet demirtas', 0.38461538461538464),
 (234, 'mehmet koleoglu', 0.38461538461538464),
 (240, 'mehmet karaoglu', 0.38461538461538464),
 (258, 'mehmet kizginel', 0.38461538461538464),
 (266, 'mehmet yildirim', 0.38461538461538464),
 (267, 'mehmet sevindir', 0.38461538461538464),
 (279, 'mehmet kayaturk', 0.38461538461538464),
 (302, 'mehmet h celebi', 0.38461538461538464),
 (325, 'mehmet kalender', 0.38461538461538464),
 (334, 'mehmet karausta', 0.38461538461538464),
 (353, 'mehmet tasdemir', 0.38461538461538464),
 (403, 'mehmet kulaksiz', 0.38461538461538464),
 (429, 'mehmet aydinlar', 0.3846153846153846